# Arash SC create a project for head position



This notebook demonstrates the necessary steps to use python and DeepLabCut for creating and analysins videos from orientation 
experiment 

This notebook illustrates how to:
 
- make a movie from images 
- create a project
- extract training frames
- label the frames
- plot the labeled images (optional)
- create a training set
- train a network
- evaluate a network
- analyze a novel video
- create an automatically labeled video (optional)
- Go to Matlab 



## Create a new project

It is always good idea to keep the projects seperate. This function creates a new project with subdirectories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos to the project at any stage of the project. 

In [2]:
import deeplabcut
## https://github.com/AlexEMG/DeepLabCut

In [3]:
task='ar21_07_06_20LR_R' # Enter the name of your experiment Task
task='ar21_08_06_20Face2' 

experimenter='Arash' # Enter the name of the experimenter
Mainfolder =  'D:\\videos\\ar21\\2020_07_08'; # Enter the name of the folder with images folders inside 
filenames = ['4.avi','6.avi','11.avi']; # Enter the name of the files you want to train
video = [str(Mainfolder)+str(f) for f in filenames]; # add path to each file
deeplabcut.create_new_project(task,experimenter,video, working_directory=Mainfolder,copy_videos=False) #change the working directory to where you want the folders created.

Created "D:\videos\ar21\2020_07_08\ar21_08_06_20Face-Arash-2020-07-15\videos"
Created "D:\videos\ar21\2020_07_08\ar21_08_06_20Face-Arash-2020-07-15\labeled-data"
Created "D:\videos\ar21\2020_07_08\ar21_08_06_20Face-Arash-2020-07-15\training-datasets"
Created "D:\videos\ar21\2020_07_08\ar21_08_06_20Face-Arash-2020-07-15\dlc-models"
Copying the videos
Generated "D:\videos\ar21\2020_07_08\ar21_08_06_20Face-Arash-2020-07-15\config.yaml"

A new project with name ar21_08_06_20Face-Arash-2020-07-15 is created at D:\videos\ar21\2020_07_08 and a configurable file (config.yaml) is stored there. Change the parameters in this file to adapt to your project's needs.
 Once you have changed the configuration file, use the function 'extract_frames' to select frames for labeling.
. [OPTIONAL] Use the function 'add_new_videos' to add new videos to your project (at any stage).


'D:\\videos\\ar21\\2020_07_08\\ar21_08_06_20Face-Arash-2020-07-15\\config.yaml'

## Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) (algo=='uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [4]:
%matplotlib inline
path_config_file = 'D:\\videos\\ar21\\2020_07_08\\ar21_08_06_20Face-Arash-2020-07-15\\config.yaml';

#Enter the path of the config file that was just created from the above step (check the folder)
deeplabcut.extract_frames(path_config_file,'automatic','uniform',crop=False) #there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper. 
#deeplabcut.extract_frames(path_config_file,'automatic','uniform',crop=True, checkcropping=True) #there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper. 
#You can change the cropping to false, then delete the checkcropping part!

Config file read successfully.

Frames were selected.
You can now label the frames using the function 'label_frames' (if you extracted enough frames for all videos).


## define the config path and load deeplabcut


In [5]:
# write the Path config file path
path_config_file = 'E:\\movies\\ar2breathing\\10_08_19\\ar2breathing_10_08_19LR_R-Arash-2020-04-30\\config.yaml';
path_config_file = 'D:\\videos\\ar21\\2020-07-06\\ar21_07_06_20Face2-Arash-2020-07-07\\config.yaml';


## Label the extracted frames
Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [7]:
%gui wx

deeplabcut.label_frames(path_config_file)

RuntimeError: wrapped C/C++ object of type DirDialog has been deleted

**Check the labels**

Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check\_labels'  to do so. It is used as follows:

In [ ]:

deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

## Reminder: Build your skeleton connections before you create a training set!


In [ ]:
deeplabcut.SkeletonBuilder(config_path)

If the labels need adjusted, you can use the refinement GUI to move them around! Check that out below.

## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

## Start training  
### If yu want to use your GPU, you need to exit here and either work from the Docker container, your own TensorFlow installation in an Anaconda env

This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
deeplabcut.train_network(path_config_file,shuffle=1,displayiters=1000,saveiters=1000)

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [14]:
deeplabcut.evaluate_network(path_config_file,Shuffles=[1], plotting=False)

Config:
{'all_joints': [[0], [1]],
 'all_joints_names': ['nose', 'snout'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_ar21_07_06_20Face2Jul7\\ar21_07_06_20Face2_Arash95shuffle1.mat',
 'dataset_type': 'default',
 'deconvolutionstride': 2,
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\Tennessee\\Anaconda3\\envs\\dlc-windowsGPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_ar21_07_06_20Face2Jul7\\

Running  DLC_resnet50_ar21_07_06_20Face2Jul7shuffle1_821000  with # of trainingiterations: 821000
Initializing ResNet
INFO:tensorflow:Restoring parameters from D:\videos\ar21\2020-07-06\ar21_07_06_20Face2-Arash-2020-07-07\dlc-models\iteration-0\ar21_07_06_20Face2Jul7-trainset95shuffle1\train\snapshot-821000
Analyzing data...


50it [00:02, 19.50it/s]


Done and results stored for snapshot:  snapshot-821000
Results for 821000  training iterations: 95 1 train error: 1.52 pixels. Test error: 2.75  pixels.
With pcutoff of 0.1  train error: 1.52 pixels. Test error: 2.75 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [5]:

path_config_file = 'D:\\rat_movies_SC\\ar19muscimol\\10_10_2019\\ar19muscimol10-10-19_LR_R-Arash-2020-02-28\\config.yaml';
#path_config_file = 'D:\\rat_movies_SC\\ar19muscimol\\10_10_2019\\ar19muscimol10-10-19_LR_L-Arash-2020-03-01\\config.yaml';
#path_config_file = 'E:\\movies\\ar2breathing\\10_08_19\\ar2breathing_10_08_19-Arash-2020-03-26\\config.yaml'
path_config_file = 'E:\\movies\\ar2breathing\\10_08_19\\ar2breathing_10_08_19LR_R-Arash-2020-04-30\\config.yaml'
path_config_file = 'D:\\videos\\ar21\\2020-07-06\\ar21_07_06_20Face2-Arash-2020-07-07\\config.yaml';

import os

Mainfolder = 'E:\\movies\\ar2breathing\\10_06_19'
Mainfolder = 'D:\\rat_movies_SC\\ar19muscimol\\10_10_2019'
Mainfolder = 'D:\\rat_movies_SC\\ar19tear19salin\\10_08_19'
Mainfolder = 'E:\\movies\\MUSCIMOL\\ar19tear19salin\\10_08_19'
Mainfolder = 'D:\\videos\\ar21\\2020_07_08_ 215805'
text_files = [os.path.join(Mainfolder,f) for f in os.listdir(Mainfolder) if f.endswith('.mp4') and not f.endswith('L.avi') and not f.endswith('R.avi') and not f.endswith('videopoints.avi') and not f.endswith('videopoints.avi')]
#text_files = [os.path.join(Mainfolder,f) for f in os.listdir(Mainfolder) if f.endswith('R.avi') and not f.endswith('L.avi') and not f.endswith('videopoints.avi') and not f.endswith('videopoints.avi')]
#text_files = [os.path.join(Mainfolder,f) for f in os.listdir(Mainfolder) if f.endswith('L.avi') and not f.endswith('R.avi') and not f.endswith('videopoints.avi') and not f.endswith('videopoints.avi')]

#text_files = [os.path.join(Mainfolder,f) for f in os.listdir(Mainfolder) if f.endswith('.mp4') and not f.endswith('L.avi') and not f.endswith('R.avi') and not f.endswith('videopoints.avi') and not f.endswith('videopoints.avi')]

#path_config_file = 'F:\\ar19muscimol500ug500nlrightside\\10_10_19\\ar19_10102019-Fassihi-2019-11-12\\config.yaml';


#path_config_file = 'D:\\rat_movies_SC\\ar19muscimol\\10_10_2019\\ar19muscimol10-10-19_LR_L-Arash-2020-03-01\\config.yaml'
##text_files = ['']
#text_files = 'E:\\movies\\ar2breathing\\10_08_19\\20-18-27.avi'

print(text_files)
#deeplabcut.analyze_videos(path_config_file,text_files[1:len(text_files)],shuffle=1, save_as_csv=True)
#deeplabcut.analyze_videos(path_config_file,text_files[3],shuffle=1, save_as_csv=True)
deeplabcut.analyze_videos(path_config_file,text_files,shuffle=1, save_as_csv=True)

['D:\\videos\\ar21\\2020_07_08_ 215805\\10video.mp4', 'D:\\videos\\ar21\\2020_07_08_ 215805\\11video.mp4', 'D:\\videos\\ar21\\2020_07_08_ 215805\\12video.mp4', 'D:\\videos\\ar21\\2020_07_08_ 215805\\13video.mp4', 'D:\\videos\\ar21\\2020_07_08_ 215805\\14video.mp4', 'D:\\videos\\ar21\\2020_07_08_ 215805\\15video.mp4', 'D:\\videos\\ar21\\2020_07_08_ 215805\\16video.mp4', 'D:\\videos\\ar21\\2020_07_08_ 215805\\17video.mp4', 'D:\\videos\\ar21\\2020_07_08_ 215805\\18video.mp4', 'D:\\videos\\ar21\\2020_07_08_ 215805\\19video.mp4', 'D:\\videos\\ar21\\2020_07_08_ 215805\\1video.mp4', 'D:\\videos\\ar21\\2020_07_08_ 215805\\20video.mp4', 'D:\\videos\\ar21\\2020_07_08_ 215805\\21video.mp4', 'D:\\videos\\ar21\\2020_07_08_ 215805\\22video.mp4', 'D:\\videos\\ar21\\2020_07_08_ 215805\\23video.mp4', 'D:\\videos\\ar21\\2020_07_08_ 215805\\24video.mp4', 'D:\\videos\\ar21\\2020_07_08_ 215805\\25video.mp4', 'D:\\videos\\ar21\\2020_07_08_ 215805\\26video.mp4', 'D:\\videos\\ar21\\2020_07_08_ 215805\\27video

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from D:\videos\ar21\2020-07-06\ar21_07_06_20Face2-Arash-2020-07-07\dlc-models\iteration-0\ar21_07_06_20Face2Jul7-trainset95shuffle1\train\snapshot-821000


...

410it [00:12, 29.33it/s]                                                                                               

Detected frames:  400


410it [00:12, 33.31it/s]


Saving results in D:\videos\ar21\2020_07_08_ 215805...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!


'DLC_resnet50_ar21_07_06_20Face2Jul7shuffle1_821000'

## Track non tracked files


In [11]:
# Track non tracked files 
import os
path_config_file = 'F:\\ar2breathing\\10_06_19\\ar2_10062019_ii-Fassihi-2019-10-07\\config.yaml';

Mainfolder = 'Y:\\movies_Rat_SC_project\\ar2breathing\\10_06_19'
text_files = [os.path.join(Mainfolder,f) for f in os.listdir(Mainfolder) if f.endswith('.avi') and not f.endswith('L.avi') and not f.endswith('R.avi') and not f.endswith('videopoints.avi') and not f.endswith('breathing.avi')]
text_files2 = [os.path.join(Mainfolder,f) for f in os.listdir(Mainfolder) if f.endswith('L.avi') ]
#print(text_files)
#%whos

text_files3 = [f.replace("L.", ".") for f in text_files2 ]

one_not_two = set(text_files) - set(text_files3)
one_not_two = 'Y:\\movies_Rat_SC_project\\ar2breathing\\10_06_19\\22-25-05.avi'
print(one_not_two)
deeplabcut.analyze_videos(path_config_file,one_not_two,shuffle=1, save_as_csv=True)

Y:\movies_Rat_SC_project\ar2breathing\10_06_19\22-25-05.avi
Using snapshot-20000 for model F:\ar2breathing\10_06_19\ar2_10062019_ii-Fassihi-2019-10-07\dlc-models\iteration-1\ar2_10062019_iiOct7-trainset95shuffle1
INFO:tensorflow:Restoring parameters from F:\ar2breathing\10_06_19\ar2_10062019_ii-Fassihi-2019-10-07\dlc-models\iteration-1\ar2_10062019_iiOct7-trainset95shuffle1\train\snapshot-20000


INFO:tensorflow:Restoring parameters from F:\ar2breathing\10_06_19\ar2_10062019_ii-Fassihi-2019-10-07\dlc-models\iteration-1\ar2_10062019_iiOct7-trainset95shuffle1\train\snapshot-20000


The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!


In [ ]:
path_config_file = 'D:\\rat_movies_SC\\ar17record\\02_27_19\\sc_ar17record_02272019_3-Fassihi-2019-04-28\\config.yaml'
video=['D:\\rat_movies_SC\\ar17\\02_25_19\\22-38-53.avi']
deeplabcut.analyze_videos(path_config_file,video,shuffle=1, save_as_csv=True)

In [ ]:
import os
Mainfolder = 'D:\\rat_movies_SC\\ar17\\02_25_19'
f2 = os.listdir(Mainfolder)
text_files = [Mainfolder+'\\'+f for f in os.listdir(Mainfolder) if f.endswith('.avi') and not f.endswith('L.avi') and not f.endswith('R.avi') and not f.endswith('videopoints.avi') and not f.endswith('videopoints.avi')]
print(text_files)
      

## Extract outlier frames [This is important step actually]
This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. Make sure to provide the correct value of the "iterations" as it will be used to create the unique directory where the extracted frames will be saved.

In [4]:
#video=['D:\\rat_movies_SC\\ar17record\\02_27_19\\analyzed\\18-37-27.avi']
#video=['D:\\rat_movies_SC\\ar15record\\02_27_19\\17-33-50video.mp4']
#video=['D:\\Dropbox\\ar17\\02_25_19\\22-37-23.avi']
video = ['D:\\movies_Rat_SC_project\\ar1\\05_24_19\\11-09-14.avi']
video = ['D:\\movies_Rat_SC_project\\ar20\\06_20_19\\17-19-45.avi','D:\\movies_Rat_SC_project\\ar20\\06_20_19\\17-31-05.avi','D:\\movies_Rat_SC_project\\ar20\\06_20_19\\17-30-40.avi']
#15-17-08v
video = ['D:\\rat_movies_SC\\ar19tear19salin\\10_08_19\\00-27-18.avi','D:\\rat_movies_SC\\ar19tear19salin\\10_08_19\\00-24-47.avi']
video = ['D:\\rat_movies_SC\\ar19tear19salin\\10_08_19\\00-24-26video.mp4']
video = ['E:\\movies\\ar2breathing\\10_06_19\\21-57-55L.avi']
video = ['E:\\movies\\ar2breathing\\10_08_19\\20-18-27R.avi']

#path_config_file = 'D:\\rat_movies_SC\\ar19muscimol\\10_10_2019\\ar19muscimol10-10-19_LR_L-Arash-2020-03-01\\config.yaml';

#path_config_file = 'D:\\rat_movies_SC\\ar17record\\02_27_19\\sc_ar17record_02272019_3-Fassihi-2019-04-28\\config.yaml'
#path_config_file = 'D:\\rat_movies_SC\\ar15record\\02_27_19\\sc_ar15record_02272019-Fassihi-2019-03-12\\config.yaml'
#path_config_file = 'D:\\movies_Rat_SC_project\\ar1\\05_24_19\\sc_aa1_05242019-Fassihi-2019-05-30\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)
#path_config_file = 'D:\\movies_Rat_SC_project\\ar3\\06_07_19\\sc_aa3_06072019-Fassihi-2019-06-10\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)
#path_config_file = 'D:\\movies_Rat_SC_project\\ar14\\06_18_19\\sc_ar14_06182019-Fassihi-2019-06-19\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)
print(path_config_file)

#deeplabcut.extract_outlier_frames(path_config_file,video,outlieralgorithm='uncertain',p_bound=.6)
#deeplabcut.extract_outlier_frames(path_config_file,video,outlieralgorithm='uncertain',comparisonbodyparts =  ['nose','snout'],p_bound=.6)
deeplabcut.extract_outlier_frames(path_config_file,video,outlieralgorithm='uncertain',p_bound=.9)

E:\movies\ar2breathing\10_08_19\ar2breathing_10_08_19LR_R-Arash-2020-04-30\config.yaml
network parameters: DeepCut_resnet50_ar2breathing_10_08_19LR_RApr30shuffle1_163000
Method  uncertain  found  172  putative outlier frames.
Do you want to proceed with extracting  20  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, p_bound, comparisonbodyparts) or use a different method.
yes/noyes
Loading video...
Duration of video [s]:  2.4 , recorded @  200.0 fps!
Overall # of frames:  480 with (cropped) frame dimensions: 
Kmeans-quantization based extracting of frames from 0.0  seconds to 2.4  seconds.
Extracting and downsampling... 171  frames from the video.


171it [00:00, 744.13it/s]


Kmeans clustering ... (this might take a while)
Let's select frames indices: [472, 18, 184, 21, 94, 318, 180, 20, 216, 47, 81, 190, 100, 198, 41, 12, 228, 83, 44, 477]
Creating the symbolic link of the video
AUTOMATIC ADDING OF VIDEO TO CONFIG FILE FAILED! You need to do this manually for including it in the config.yaml file!
Videopath: E:\movies\ar2breathing\10_08_19\20-18-27R.avi Coordinates for cropping: None
The outlier frames are extracted. They are stored in the subdirectory labeled-data\20-18-27R.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.


In [12]:
print(video[2])


D:\movies_Rat_SC_project\ar20\06_20_19\17-30-40avi


In [4]:

path_config_file = 'D:\\rat_movies_SC\\ar19\\02_07_19\\sc_ar19_02072019-Fassihi4-2019-02-13\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)

import os
Mainfolder = 'D:\\rat_movies_SC\\ar19\\02_07_19'


f = os.listdir(Mainfolder)
text_files = [f for f in os.listdir(Mainfolder) if f.endswith('.avi')]


my_list = text_files
thisApen = Mainfolder+'kir'
thisApen = thisApen.replace("kir", "\\")
string = thisApen
my_new_list = [ string + x for x in my_list]

deeplabcut.extract_outlier_frames(path_config_file,my_new_list)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\rat_movies_SC\\ar19\\02_07_19'

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset.

In [5]:
#path_config_file = 'D:\\movies_Rat_SC_project\\ar1\\05_24_19\\sc_aa1_05242019-Fassihi-2019-05-30\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)
#path_config_file = 'D:\\movies_Rat_SC_project\\ar3\\06_07_19\\sc_aa3_06072019-Fassihi-2019-06-10\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)

#import matplotlib
#print(matplotlib.__version__)
%gui wx
#path_config_file = 'D:\\rat_movies_SC\\ar15record\\02_27_19\\sc_ar15record_02272019-Fassihi-2019-03-12\\config.yaml'
#print(path_config_file)
deeplabcut.refine_labels(path_config_file)

ERROR:root:Cannot activate multiple GUI eventloops


Windows
Checking labels if they are outside the image
Checking labels if they are outside the image


ValueError: On level 0, label max (1) >= length of level  (1). NOTE: this index is in an inconsistent state

Closing... The refined labels are stored in a subdirectory under labeled-data. Use the function 'merge_datasets' to augment the training dataset, and then re-train a network using create_training_dataset followed by train_network!


In [8]:
#Once all folders are relabeled, check them and advance. See how to check labels, above!
#path_config_file = 'D:\\movies_Rat_SC_project\\ar1\\05_24_19\\sc_aa1_05242019-Fassihi-2019-05-30\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)
#path_config_file = 'D:\\movies_Rat_SC_project\\ar3\\06_07_19\\sc_aa3_06072019-Fassihi-2019-06-10\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)

deeplabcut.merge_datasets(path_config_file)

Merged data sets and updated refinement iteration to 2.
Now you can create a new training set for the expanded annotated images (use create_training_dataset).


## Create a new iteration of training dataset [optional step]
Following the refine labels, append these frames to the original dataset to create a new iteration of training dataset.

In [9]:
list_of_ints = list(range(2))
print(list_of_ints)
deeplabcut.create_training_dataset(path_config_file,Shuffles= list_of_ints) ;

[0, 1]
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [9]:
print(path_config_file)
videofile  = ['D:\\rat_movies_SC\\ar19tear19salin\\10_08_19\\00-24-02.avi','D:\\rat_movies_SC\\ar19tear19salin\\10_08_19\\00-24-47.avi']
videofile = ['F:\\ar2breathing\\10_18_19\\00-16-30video.mp4']
videofile = ['F:\\ar2breathing\\10_18_19\\00-16-30video.mp4']
videofile = ['D:\\rat_movies_SC\\ar2breathing\\10_06_19\\Right\\22-17-49R.avi']

deeplabcut.create_labeled_video(path_config_file,videofile,save_frames=True) # my_new_list was created in prevouse cell as all videos in the folder 
#deeplabcut.create_labeled_video(path_config_file,text_files,save_frames=True) # my_new_list was created in prevouse cell as all videos in the folder 

D:\rat_movies_SC\ar2breathing\10_06_19\ar19salin_10-06-19_LR-Arash-2020-01-24\config.yaml
Starting %  D:\rat_movies_SC\ar2breathing\10_06_19\Right ['D:\\rat_movies_SC\\ar2breathing\\10_06_19\\Right\\22-17-49R.avi']
Loading  D:\rat_movies_SC\ar2breathing\10_06_19\Right\22-17-49R.avi and data.
False 0 202 0 181
562
Duration of video [s]:  2.81 , recorded with  200.0 fps!
Overall # of frames:  562 with cropped frame dimensions:  202 181
Generating frames and creating video.


100%|████████████████████████████████████████████████████████████████████████████████| 562/562 [02:39<00:00,  2.67it/s]


All labeled frames were created, now generating video...


## Plot the trajectories of the analyzed videos (optional)
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,my_new_list)

